# 🔬 Feature Analysis & Quality Check

Ce notebook analyse les features créées pour:
1. Vérifier la qualité des features
2. Analyser les distributions
3. Identifier les features les plus importantes
4. Détecter les corrélations
5. Préparer pour le modeling


Cellule 2 — Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports successful")


✅ Imports successful


Cellule 3 — Markdown
## 1. Charger les Features

Cellule 4 — Chargement Features

In [3]:
# Charger les features
features_path = Path('../data/processed/v1/features.parquet')
df = pd.read_parquet(features_path)

print(f"📊 Dataset Shape: {df.shape}")
print(f"   Records: {len(df):,}")
print(f"   Features: {len(df.columns)}")
print(f"\n📅 Seasons: {df['season'].unique()}")
print(f"   Total: {df['season'].nunique()} seasons")
print(f"\n⚽ Teams: {df['team'].nunique()} unique teams")


📊 Dataset Shape: (6076, 44)
   Records: 6,076
   Features: 44

📅 Seasons: ['2015-2016' '2016-2017' '2017-2018' '2018-2019' '2019-2020' '2020-2021'
 '2021-2022' '2022-2023']
   Total: 8 seasons

⚽ Teams: 32 unique teams


Cellule 5 — Aperçu

In [4]:
# Aperçu des données
df.head(10)


,season,team,gameweek,current_position,current_points,matches_played,wins,draws,losses,goals_for,...,target_final_position,form_last_5_goal_diff,avg_total_scoring_att,avg_ontarget_scoring_att,avg_total_pass,avg_accurate_pass,avg_total_tackle,avg_won_contest,avg_saves,avg_total_offside
0,2015-2016,Manchester City,1,1,3,1,1,0,0,3.0,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-2016,Manchester City,2,1,6,2,2,0,0,6.0,...,4,3.0,20.000000,7.000000,764.000000,693.000000,10.000000,7.000000,2.000000,NaN
2,2015-2016,Manchester City,3,1,9,3,3,0,0,8.0,...,4,6.0,19.000000,7.500000,593.500000,519.000000,13.500000,10.500000,2.500000,1.000000
3,2015-2016,Manchester City,4,1,12,4,4,0,0,10.0,...,4,8.0,18.000000,8.000000,558.000000,484.000000,14.666667,10.333333,2.000000,1.500000
4,2015-2016,Manchester City,5,1,15,5,5,0,0,11.0,...,4,10.0,18.000000,7.250000,579.750000,511.500000,14.750000,11.000000,2.000000,1.333333
5,2015-2016,Manchester City,6,1,15,6,5,0,1,12.0,...,4,11.0,18.600000,7.200000,563.400000,489.600000,14.800000,9.600000,2.000000,1.333333
6,2015-2016,Manchester City,7,2,15,7,5,0,2,13.0,...,4,7.0,20.000000,7.333333,581.000000,507.333333,15.500000,10.833333,1.800000,1.500000
7,2015-2016,Manchester City,8,1,18,8,6,0,2,19.0,...,4,1.0,20.285714,7.428571,566.285714,486.714286,15.857143,10.285714,2.166667,1.800000
8,2015-2016,Manchester City,9,1,21,9,7,0,2,24.0,...,4,4.0,20.625000,7.875000,569.250000,488.125000,16.375000,10.125000,2.285714,1.666667
9,2015-2016,Manchester City,10,1,22,10,7,1,2,24.0,...,4,6.0,20.111111,8.222222,568.888889,486.111111,16.111111,10.888889,2.285714,1.714286


Cellule 6 — Metadata

In [5]:
# Charger les métadonnées
metadata_path = Path('../data/processed/v1/feature_metadata.json')
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

print("📋 Feature Metadata:")
print(f"   Created: {metadata['created_at']}")
print(f"   Form window: {metadata['form_window']}")

print("\n📊 Feature Categories:")

# Classifier les features par catégorie
feature_names = [col for col in df.columns if col not in ['season', 'team', 'gameweek', 'target_final_points', 'target_final_position']]

categories = {
    'Cumulative': [f for f in feature_names if f.startswith(('current_', 'matches_', 'wins', 'draws', 'losses', 'goals_'))],
    'Form': [f for f in feature_names if 'form_' in f or 'last_' in f],
    'Home/Away': [f for f in feature_names if f.startswith(('home_', 'away_'))],
    'Ratios': [f for f in feature_names if any(x in f for x in ['_rate', '_per_', '_ppg'])],
    'Stats': [f for f in feature_names if 'avg_' in f],
}

for cat, features in categories.items():
    print(f"   {cat}: {len(features)} features")


📋 Feature Metadata:
   Created: 2025-12-11T00:35:41.440399
   Form window: 5

📊 Feature Categories:
   Cumulative: 11 features
   Form: 5 features
   Home/Away: 6 features
   Ratios: 11 features
   Stats: 8 features
